## Введение

В 2000 году Enron была одной из крупнейших компаний США. К 2002 году он обанкротился из-за широко распространенного корпоративного мошенничества. В результате федерального расследования значительный объем обычно конфиденциальной информации стал общедоступным, включая десятки тысяч электронных писем и подробные финансовые данные для высшего руководства. 

Мошенничество с компанией Enron - это крупнейший случай корпоративного мошенничества в истории Америки. Основанная в 1985 году, корпорация Enron обанкротилась к концу 2001 года из-за широко распространенного корпоративного мошенничества и коррупции. Перед ее падением журнал Fortune шесть лет подряд называл Enron «самой инновационной компанией Америки».

Здесь мы создаем алгоритм обучения для выявления сотрудников-мошенников (далее "подозреваемые") с использованием набора данных Enron.

In [ ]:
from time import time
import pandas as pd 
import pickle
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest, f_classif   
from sklearn.model_selection import train_test_split, GridSearchCV  
                                                                 
from sklearn.model_selection import StratifiedShuffleSplit   
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np     

from sklearn.tree import DecisionTreeClassifier

import warnings

## Считавание датасета

In [ ]:
with open("dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
enron_data = pd.DataFrame.from_dict(data_dict, orient='index')

## Начальные значения в датасете

In [ ]:
enron_data

In [ ]:
print(f"Всего людей: {len(enron_data.index)}")
print(f"Подозреваемые: {enron_data['poi'].value_counts()[True]}")

In [ ]:
enron_data.fillna(0, inplace=True)
enron_data.replace(to_replace='NaN', value=0.0, inplace=True)

## Выбросы

Здесь происходит предварительная обработка данных. Слева график до очищения от выбросов, справа - после

In [ ]:
figure, (axes1, axes2) = plt.subplots(ncols=2, figsize=(12, 6))

axes1.scatter(enron_data.salary, enron_data.bonus)
axes1.set_xlabel("Salary ($)", fontsize=15)
axes1.set_ylabel("Bonus ($)", fontsize=15)
axes1.set_title("Before Outlier Removal", fontsize=15)
axes1.spines['right'].set_visible(False)
axes1.spines['top'].set_visible(False)

# Удаляем выбросы
enron_data.drop(['TOTAL'], axis = 0, inplace=True)
enron_data.drop(['THE TRAVEL AGENCY IN THE PARK'], axis = 0, inplace= True)

axes2.scatter(enron_data.salary, enron_data.bonus)
axes2.set_xlabel("Salary ($)", fontsize=15)
axes2.set_ylabel("Bonus ($)", fontsize=15)
axes2.set_title("After Outlier Removal", fontsize=15)
axes2.spines['right'].set_visible(False)
axes2.spines['top'].set_visible(False)

figure.tight_layout()
plt.show()

## Полезность свойств

Используя готовое решение, определяем свойства, имеющие основное влияние на состояние обьектов в датасете. Дальше выберем только те, у которых score >= 2, чтобы процесс обучения модели работал быстрее с минимальными потерями информации

In [ ]:
enron_data["fraction_from_poi"] = enron_data["from_poi_to_this_person"].\
                                  divide(enron_data["to_messages"], fill_value=0.0) 

enron_data["fraction_to_poi"] = enron_data["from_this_person_to_poi"].\
                                divide(enron_data["from_messages"], fill_value=0.0)
enron_data['shared_receipt_poi_ratio'] = enron_data['shared_receipt_with_poi'].\
                                 divide(enron_data['to_messages'], fill_value=0.0)

enron_data['bonus_to_salary'] = enron_data['bonus'].divide(enron_data['salary'], fill_value=0.0)
enron_data['bonus_to_total'] = enron_data['bonus'].divide(enron_data['total_payments'], fill_value=0.0)

enron_data["fraction_from_poi"] = enron_data["fraction_from_poi"].fillna(0.0)
enron_data["fraction_to_poi"] = enron_data["fraction_to_poi"].fillna(0.0)
enron_data["shared_receipt_poi_ratio"] = enron_data["shared_receipt_poi_ratio"].fillna(0.0)
enron_data["bonus_to_salary"] = enron_data["bonus_to_salary"].fillna(0.0)
enron_data["bonus_to_total"] = enron_data["bonus_to_total"].fillna(0.0)


enron_data["poi"] = enron_data["poi"].astype(float)

features_list = ['to_messages', 'deferral_payments', 'bonus_to_salary',
                 'loan_advances', 'bonus_to_total', 'restricted_stock_deferred',
                 'deferred_income', 'total_stock_value', 'expenses',
                 'from_poi_to_this_person', 'exercised_stock_options', 'from_messages',
                 'other', 'from_this_person_to_poi', 'long_term_incentive',
                 'shared_receipt_poi_ratio', 'restricted_stock', 'director_fees',
                 'fraction_from_poi', 'fraction_to_poi']

labels = enron_data["poi"].tolist()
features = enron_data[features_list].values.tolist()

# Performing feature selection
selector = SelectKBest(f_classif, k=19)
selector.fit(features, labels)

scores = -np.log10(selector.pvalues_)

# Plotting the features wrt scores
figure, axes = plt.subplots(figsize=(8,9))
axes.barh(features_list, width=scores)
axes.set_yticklabels(features_list, fontsize=14)
axes.set_xlabel("Scores")
axes.set_title("Scores of Features", fontsize=16)
axes.spines['right'].set_visible(False)
axes.spines['top'].set_visible(False)
plt.show()

In [ ]:
updated_features_list = ["bonus_to_total", "loan_advances", "bonus_to_salary", "deferred_income",
                         "total_stock_value", "exercised_stock_options", "long_term_incentive",
                         "shared_receipt_poi_ratio", "restricted_stock", "fraction_to_poi"]

labels = enron_data["poi"].tolist()
features = enron_data[features_list].values.tolist()

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=67)

## Гиперпараметры

Обучение будем проводить алгоритмом Дерево принятия решений. 
Находим лучшие для нашего датасета значения гиперпараметров, используя метод кросс-валидации.

In [ ]:
clf_parameters = { 'criterion': ['gini', 'entropy'],
                   'max_depth': [None, 1, 2, 4, 5, 10, 15, 20],
                   'min_samples_split': [2, 4, 6, 8, 10, 20, 30, 40],
                   'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 10, 20, 30] }

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=67)
clf = GridSearchCV(DecisionTreeClassifier(), param_grid = clf_parameters, cv = cv, scoring = 'f1')
clf.fit(features,labels)

best_clf_parameters = clf.best_estimator_
best_clf_parameters

## Обучение

На найденом лучшем наборе гиперпараметров обучаем нашу модель

In [ ]:
clf = clf.best_estimator_

clf.fit(features_train, labels_train)
predicitons = clf.predict(features_test)

print("Accuracy:", round(accuracy_score(labels_test, predicitons), 3))
print("Precision:", round(precision_score(labels_test, predicitons), 3))
print("Recall:", round(recall_score(labels_test, predicitons), 3))
print("F1-Score:", round(f1_score(labels_test, predicitons), 3))

## Итог

Обученая модель выдала результат 0.909 точности определения людей, которые могли быть угрозой для компании